In [ ]:
# Clean up to reduce the different formats used (string, tuple). 
# Originally moved away from tuple for everything because sets can't deal with tuples

In [1]:
import pandas as pd
import networkx as nx

class Graph(object):

    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self._graph_dict = graph_dict

    def edges(self, vertice):
        """ returns a list of all the edges of a vertice"""
        return self._graph_dict[vertice]
        
    def all_vertices(self):
        """ returns the vertices of a graph as a set """
        return set(self._graph_dict.keys())

    def all_edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self._graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self._graph_dict:
            self._graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        edge = set(edge)
        vertex1, vertex2 = tuple(edge)
        for x, y in [(vertex1, vertex2), (vertex2, vertex1)]:
            if x in self._graph_dict:
                self._graph_dict[x].append(y)
            else:
                self._graph_dict[x] = [y]

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self._graph_dict:
            for neighbour in self._graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges
    
    def __iter__(self):
        self._iter_obj = iter(self._graph_dict)
        return self._iter_obj
    
    def __next__(self):
        """ allows us to iterate over the vertices """
        return next(self._iter_obj)

    def __str__(self):
        res = "vertices: "
        for k in self._graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def find_path(self, start_vertex, end_vertex, path=None):
        """ find a path from start_vertex to end_vertex 
            in graph """
        if path == None:
            path = []
        graph = self._graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return path
        if start_vertex not in graph:
            return None
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_path = self.find_path(vertex, 
                                               end_vertex, 
                                               path)
                if extended_path: 
                    return extended_path
        return None
    
    
    def find_all_paths(self, start_vertex, end_vertex, path=[]):
        """ find all paths from start_vertex to 
            end_vertex in graph """
        graph = self._graph_dict 
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return [path]
        if start_vertex not in graph:
            return []
        paths = []
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_paths = self.find_all_paths(vertex, 
                                                     end_vertex, 
                                                     path)
                for p in extended_paths: 
                    paths.append(p)
        return paths
    
def get_edges(p): 
    
    # get a list of all the steps around the position 
    next_steps = [(p[0],p[1]-1),(p[0]-1,p[1]),(p[0]+1,p[1]),(p[0],p[1]+1)]

    
    # screen out where the next steps falls off the edge of the map
    next_steps = [x for x in next_steps if (x[0] >= 0 and x[0] < len(df.columns) and x[1] >= 0 and x[1] < len(df))]

    
    # for each next step, filter out to just include those one letter higher or less
    next_steps = [y for y in next_steps if 
                  ((df.loc[p[1],p[0]] == 'S')
                    or 
                  ((ord(df.loc[y[1],y[0]]) == ord(df.loc[p[1],p[0]])+1 
                   or 
                   ord(df.loc[y[1],y[0]]) == ord(df.loc[p[1],p[0]]))
                   and 
                   df.loc[y[1],y[0]] not in ('S')))]

    return next_steps

workdir = 'C:\\Users\\ed.deane\\OneDrive - Sabio Ltd\\Documents\\Customers\\IKEA\\Knowledge\\Python\\AdventOfCode\\Day 12 Hill Climbing Algorithm\\'

df= pd.read_csv(workdir+'test_input.txt', header=None)

# For each line of the file, create a row and column value corresponding to that in the file
for index, row in df.iterrows():
    j=0
    for i in str(row[0]):
        df.at[index,j]=i
        j+=1
        
# get the start and end position
start = df.where(df=='S').dropna(how='all').dropna(axis=1)
start = (start.columns[0],list(start.index)[0])
end = df.where(df=='E').dropna(how='all').dropna(axis=1)
end = (end.columns[0],list(end.index)[0]) 


In [2]:
# Go through each field and initialise the graph with its row,column as a tuple (row,column)
# An edge is created where the adjoining field (nsew) is either the same value or one more
g = {}
for index, row in df.iterrows():
    for j in range(len(df.columns)):
        g[(j,index)] = get_edges((j,index))

In [5]:
graph = Graph(g)

print("Vertices of graph:")
print(graph.all_vertices())

print("Edges of graph:")
print(graph.all_edges())


print('The paths from vertex start to vertex end:')
path = graph.find_all_paths(start,end)
print(path)

Vertices of graph:
{(4, 0), (3, 4), (4, 3), (3, 1), (5, 4), (5, 1), (0, 2), (2, 2), (1, 0), (1, 3), (7, 4), (6, 2), (7, 1), (4, 2), (3, 0), (3, 3), (5, 0), (5, 3), (0, 1), (2, 4), (1, 2), (0, 4), (2, 1), (6, 1), (7, 0), (6, 4), (7, 3), (3, 2), (4, 1), (5, 2), (4, 4), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4), (2, 3), (7, 2), (6, 0), (6, 3)}
Edges of graph:
[{(1, 0), (0, 0)}, {(0, 1), (0, 0)}, {(1, 0), (2, 0)}, {(1, 0), (1, 1)}, {(2, 0), (2, 1)}, {(3, 1), (3, 0)}, {(4, 0), (3, 0)}, {(4, 0), (5, 0)}, {(5, 0), (6, 0)}, {(7, 0), (6, 0)}, {(0, 1), (1, 1)}, {(0, 1), (0, 2)}, {(1, 1), (2, 1)}, {(1, 1), (1, 2)}, {(2, 1), (2, 2)}, {(3, 1), (3, 2)}, {(4, 1), (4, 2)}, {(4, 1), (5, 1)}, {(6, 1), (5, 1)}, {(6, 1), (6, 2)}, {(7, 0), (7, 1)}, {(0, 2), (0, 3)}, {(1, 2), (2, 2)}, {(1, 2), (1, 3)}, {(2, 3), (2, 2)}, {(3, 2), (3, 3)}, {(7, 1), (7, 2)}, {(0, 3), (0, 4)}, {(2, 3), (1, 3)}, {(2, 3), (2, 4)}, {(3, 3), (4, 3)}, {(5, 3), (4, 3)}, {(5, 3), (6, 3)}, {(6, 2), (6, 3)}, {(7, 2), (7, 3)}, {(0, 4), (1, 

In [6]:
l= 1000000000000000000
for p in path:
    if (len(p) < l):
        l = len(p)
l

31

In [4]:
end = (4,2)

In [8]:
g.

{(0, 0): [(1, 0), (0, 1)],
 (1, 0): [(2, 0), (1, 1)],
 (2, 0): [(2, 1)],
 (3, 0): [(3, 1)],
 (4, 0): [(3, 0)],
 (5, 0): [(4, 0)],
 (6, 0): [(5, 0)],
 (7, 0): [(6, 0)],
 (0, 1): [(1, 1), (0, 2)],
 (1, 1): [(2, 1), (1, 2)],
 (2, 1): [(2, 2)],
 (3, 1): [(3, 2)],
 (4, 1): [(4, 2)],
 (5, 1): [(4, 1), (6, 1)],
 (6, 1): [(5, 1), (6, 2)],
 (7, 1): [(7, 0)],
 (0, 2): [(0, 1), (0, 3)],
 (1, 2): [(2, 2), (1, 3)],
 (2, 2): [(2, 1), (1, 2), (2, 3)],
 (3, 2): [(3, 3)],
 (4, 2): [],
 (5, 2): [],
 (6, 2): [(6, 1)],
 (7, 2): [(7, 1)],
 (0, 3): [(0, 2), (0, 4)],
 (1, 3): [(1, 2), (2, 3)],
 (2, 3): [(2, 2), (1, 3), (2, 4)],
 (3, 3): [(4, 3)],
 (4, 3): [(5, 3)],
 (5, 3): [(6, 3)],
 (6, 3): [(6, 2)],
 (7, 3): [(7, 2)],
 (0, 4): [(0, 3), (1, 4)],
 (1, 4): [(1, 3)],
 (2, 4): [(3, 4)],
 (3, 4): [(4, 4)],
 (4, 4): [(5, 4)],
 (5, 4): [(6, 4)],
 (6, 4): [(7, 4)],
 (7, 4): [(7, 3)]}

In [7]:

print(nx.shortest_path(g, source=start, target=end))

AttributeError: 'dict' object has no attribute 'is_directed'